In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ES_HOST = os.getenv("ES_HOST")
ES_USER_ID = os.getenv("ES_ID")
ES_PASSWD = os.getenv("ES_PWD")
ES_INDEX_NAME = os.getenv("ES_INDEX_NAME")
EMBEDDING_BASE_URL=os.getenv("EMBEDDING_BASE_URL")
EMBEDDING_MODEL=os.getenv("EMBEDDING_MODEL")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGUNGFACE_TOKEN")

In [25]:
# from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
# from langchain_community.vectorstores.elastic_vector_search import ElasticKnnSearch
# %pip install lark
from langchain_elasticsearch import ElasticsearchStore, DenseVectorStrategy, ElasticsearchEmbeddings
from langchain_elasticsearch import ElasticsearchRetriever
from elasticsearch import Elasticsearch
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain_openai import ChatOpenAI
from langchain.retrievers import MultiQueryRetriever, SelfQueryRetriever
from pydantic.v1 import BaseModel, Field  # <-- Uses v1 namespace
from langchain.tools import StructuredTool

In [28]:
embeddings = HuggingFaceHubEmbeddings(model=EMBEDDING_BASE_URL, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHubEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpointEmbeddings`.
  warn_deprecated(


In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate("Tell me something")

In [5]:
chain = prompt | llm 

In [6]:
chain.invoke({})

AIMessage(content='Tell me something interesting!', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 88, 'total_tokens': 93}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_c9aa9c0491', 'finish_reason': 'stop', 'logprobs': None}, id='run-44bf2768-b381-4390-817c-8be79cce1c28-0', usage_metadata={'input_tokens': 88, 'output_tokens': 5, 'total_tokens': 93})

In [7]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

In [9]:
chain.invoke({})

'Tell me something interesting!'

# apply topic

In [10]:
prompt = ChatPromptTemplate.from_template("Tell me something about {topic}")

In [11]:
chain = prompt | llm | StrOutputParser()

In [12]:
chain.invoke({"topic":"sora develop by sora"})

'It seems like you might be referring to "Sora" in the context of software development or a specific project. However, "Sora" can refer to several different things depending on the context:\n\n1. **Sora (OverDrive)**: Sora is an app developed by OverDrive, Inc. that provides access to eBooks and audiobooks for students. It is designed for educational purposes, allowing students to borrow digital content from their school\'s library.\n\n2. **Sora (Kingdom Hearts)**: Sora is also the main protagonist in the "Kingdom Hearts" video game series developed by Square Enix. This character is not related to software development but is well-known in the gaming community.\n\n3. **Sora (WebRTC)**: Sora is a WebRTC SFU (Selective Forwarding Unit) developed by Shiguredo Inc. It is used for real-time communication applications, enabling efficient video and audio streaming.\n\n4. **Sora (Programming Language)**: There is also a lesser-known programming language named Sora, which is designed for specifi

# 체인 분해

In [13]:
prompt_formatter = prompt.invoke({"topic":"sora"})
prompt_formatter


ChatPromptValue(messages=[HumanMessage(content='Tell me something about sora')])

In [15]:
llm_output = llm.invoke(prompt_formatter)

In [16]:
parser = StrOutputParser()
parser.invoke(llm_output)

'Sora is a character from the popular video game series "Kingdom Hearts," which is developed and published by Square Enix in collaboration with Disney. Sora is the main protagonist of the series and is known for his cheerful and optimistic personality. He wields a weapon called the Keyblade, which has the power to combat creatures known as the Heartless and to lock and unlock various worlds\' keyholes, preventing them from being consumed by darkness.\n\nSora\'s journey begins on the Destiny Islands, where he lives with his friends Riku and Kairi. When their world is attacked by the Heartless, Sora is separated from his friends and finds himself in a place called Traverse Town. There, he meets Donald Duck and Goofy, who are on a mission to find their missing king, Mickey Mouse. Together, they travel across various Disney-themed worlds, battling the forces of darkness and searching for Sora\'s lost friends.\n\nThroughout the series, Sora\'s character grows and evolves as he faces numerou

# 1회성 사용

In [17]:
(prompt|llm|parser).invoke({"topic":"llm"})

'Sure! LLM can refer to different things depending on the context, but two of the most common meanings are:\n\n1. **Master of Laws (LL.M.)**:\n   - **Definition**: The LL.M. is an advanced, postgraduate academic degree in law. It stands for "Legum Magister" in Latin, which translates to "Master of Laws."\n   - **Purpose**: This degree is typically pursued by individuals who already hold a primary law degree (such as a J.D. in the United States or an LL.B. in many other countries) and wish to specialize in a particular area of law, such as international law, tax law, human rights law, or intellectual property law.\n   - **Duration**: The program usually lasts one year if pursued full-time, though part-time options are often available.\n   - **Global Recognition**: The LL.M. is recognized internationally and can enhance career prospects, provide opportunities for academic research, and sometimes fulfill requirements for practicing law in a different jurisdiction.\n\n2. **Large Language M

In [38]:
es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, timeout=120)
vector_store = ElasticsearchStore(
    embedding=embeddings,
    index_name=ES_INDEX_NAME,
    es_connection=es,
    strategy=DenseVectorStrategy(hybrid=True),
)

/var/folders/jy/5_rl6575533638lskdr9s6s00000gn/T/ipykernel_9143/1668103102.py:1: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, timeout=120)
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://192.168.0.10:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [44]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [39]:
retriever = vector_store.as_retriever()

In [56]:
retriever.invoke("삼성전자")

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[Document(metadata={'id': 611501, 'hash_key': '1226b4a4c70e7bc5fee64513bdc3ef81', 'title': '이재용, 리창 중국 총리 면담…삼성 중국 사업 지원 감사 뜻 전달', 'created_date': '2024-05-26T22:45:00', 'portal': 'daum', 'media': 'SBS', 'url': 'https://v.daum.net/v/20240526224500535', 'image_url': 'https://img1.daumcdn.net/thumb/S1200x630/?fname=https://t1.daumcdn.net/news/202405/26/sbsi/20240526224502831pvnt.jpg'}, page_content='삼성 경영진은 전영현 삼성전자 DS부문장 부회장, 노태문 삼성전자 DX부문 MX사업부장 사장, 박학규 삼성전자 경영지원실장, 최윤호 삼성SDI 대표이사 사장, 최주선 삼성디스플레이 대표이사 사장, 양걸 삼성전자 삼성차이나 사장, 김원경 삼성전자 글로벌공공업무실장 사장 등이 참석했습니다.\n(사진=삼성전자 제공, 연합뉴스)\n유덕기 기자 dkyu@sbs.co.kr\nCopyright © Copyright ⓒ SBS. All rights reserved. 무단 전재, 재배포 및 AI학습 이용 금지'),
 Document(metadata={'id': 526976, 'hash_key': '2042b04cdae7634af7172c4557948636', 'title': '삼성전자 8만원대 안착 시도', 'created_date': '2024-03-28T17:00:00', 'portal': 'naver', 'media': '연합뉴스 | 네이버', 'url': 'https://n.news.naver.com/mnews/article/001/0014596368?rc=N&ntype=RANKING&sid=101', 'image_url': 'https://imgnews.pstat

In [61]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
def merge_docs(retrived_docs):
    return "\n\n".join(d.page_content for d in retrived_docs)

In [62]:
(retriever | merge_docs | llm | parser).invoke("삼성전자")

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


"삼성전자의 최근 경영 활동과 주가 동향에 대한 다양한 기사가 보도되었습니다. 주요 내용은 다음과 같습니다:\n\n1. **삼성전자 경영진 참석 행사**:\n   - 전영현 삼성전자 DS부문장 부회장, 노태문 삼성전자 DX부문 MX사업부장 사장, 박학규 삼성전자 경영지원실장, 최윤호 삼성SDI 대표이사 사장, 최주선 삼성디스플레이 대표이사 사장, 양걸 삼성전자 삼성차이나 사장, 김원경 삼성전자 글로벌공공업무실장 사장 등이 참석한 행사에 대한 보도가 있었습니다.\n\n2. **삼성전자 주가 동향**:\n   - 2024년 3월 28일 기준으로 삼성전자의 주가는 7만8800원에 거래되고 있으며, 장중 최고가는 7만9400원으로 52주 신고가를 기록했습니다.\n   - 메리츠증권은 삼성전자의 목표주가를 기존 9만4000원에서 9만5000원으로 상향 조정하고, 투자의견을 '매수'로 유지했습니다. 이는 삼성전자의 4분기 영업이익이 3분기 대비 개선될 것으로 전망되기 때문입니다.\n\n3. **삼성전자 정기주주총회**:\n   - 한종희 삼성전자 부회장은 경기 수원시 수원컨벤션센터에서 열린 제55기 삼성전자 정기주주총회에서 주주가치 제고와 주주환원 정책에 대해 언급했습니다. 2023년 기준으로 연간 9조8천억원의 배당을 지급할 계획이라고 밝혔습니다.\n\n4. **메모리 및 반도체 부문 전망**:\n   - 메리츠증권의 김선우 연구원은 삼성전자의 4분기 영업이익이 전분기 대비 소폭 개선될 것으로 전망했습니다. 특히, 반도체(DS) 부문 영업손실이 감소하고, D램 감산폭을 줄이는 등의 조치로 인해 수익성 회복이 예상된다고 밝혔습니다.\n\n이와 같은 정보는 삼성전자의 경영 전략과 주가 전망에 대한 중요한 인사이트를 제공합니다. 주주와 투자자들은 이러한 정보를 바탕으로 투자 결정을 내릴 수 있을 것입니다."

# RAG Chain

In [48]:
from langchain_core.runnables import RunnableParallel

In [65]:
chain_parallel = RunnableParallel({"context":retriever, "llm":llm })
chain_parallel

{
  context: VectorStoreRetriever(tags=['ElasticsearchStore', 'HuggingFaceHubEmbeddings'], vectorstore=<langchain_elasticsearch.vectorstores.ElasticsearchStore object at 0x14e0887c0>),
  llm: ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x14fd4c340>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x14fd4eb50>, model_name='gpt-4o', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')
}

In [66]:
chain_parallel.invoke("삼성전자")

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'context': [Document(metadata={'id': 611501, 'hash_key': '1226b4a4c70e7bc5fee64513bdc3ef81', 'title': '이재용, 리창 중국 총리 면담…삼성 중국 사업 지원 감사 뜻 전달', 'created_date': '2024-05-26T22:45:00', 'portal': 'daum', 'media': 'SBS', 'url': 'https://v.daum.net/v/20240526224500535', 'image_url': 'https://img1.daumcdn.net/thumb/S1200x630/?fname=https://t1.daumcdn.net/news/202405/26/sbsi/20240526224502831pvnt.jpg'}, page_content='삼성 경영진은 전영현 삼성전자 DS부문장 부회장, 노태문 삼성전자 DX부문 MX사업부장 사장, 박학규 삼성전자 경영지원실장, 최윤호 삼성SDI 대표이사 사장, 최주선 삼성디스플레이 대표이사 사장, 양걸 삼성전자 삼성차이나 사장, 김원경 삼성전자 글로벌공공업무실장 사장 등이 참석했습니다.\n(사진=삼성전자 제공, 연합뉴스)\n유덕기 기자 dkyu@sbs.co.kr\nCopyright © Copyright ⓒ SBS. All rights reserved. 무단 전재, 재배포 및 AI학습 이용 금지'),
  Document(metadata={'id': 526976, 'hash_key': '2042b04cdae7634af7172c4557948636', 'title': '삼성전자 8만원대 안착 시도', 'created_date': '2024-03-28T17:00:00', 'portal': 'naver', 'media': '연합뉴스 | 네이버', 'url': 'https://n.news.naver.com/mnews/article/001/0014596368?rc=N&ntype=RANKING&sid=101', 'image_url': 'https://

In [68]:
from langchain_core.runnables import RunnablePassthrough
chain_path = RunnableParallel({"context":retriever, "query":RunnablePassthrough()})


In [69]:
chain_path.invoke("LG전자")

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'context': [Document(metadata={'id': 139435, 'hash_key': '0292a05898913538d88b16c881c5839a', 'title': 'LG전자 베스트샵 서울양평220점, 그랜드 오픈 기념 사전행사 전개', 'created_date': '2023-12-21T13:54:13', 'portal': 'daum', 'media': '디지털타임스', 'url': 'https://v.daum.net/v/20231221135413058', 'image_url': 'https://img1.daumcdn.net/thumb/S1200x630/?fname=https://t1.daumcdn.net/news/202312/21/dt/20231221135414670zpbc.jpg'}, page_content="LG전자 베스트샵 서울양평220점, 그랜드 오픈 기념 사전행사 전개\nLG전자 베스트샵 서울양평점이 내년 1월 5일 'LG전자 베스트샵 서울양평220점'으로 새롭게 오픈한다고 21일 밝혔다.\n이에 LG전자 베스트샵 서울양평220점에서는 다양한 사전 행사를 진행한다.\nLG전자 베스트샵 서울양평220점의 사전 예약 행사 기간은 1차 15일부터 24일까지, 2차 25일부터 내년 1월 4일까지다.\n더불어 LG전자 베스트샵 서울양평220점 2층에서는 다양한 볼거리 및 놀거리가 준비되어 있다.\n이 글자크기로 변경됩니다.\n(예시) 가장 빠른 뉴스가 있고 다양한 정보, 쌍방향 소통이 숨쉬는 다음뉴스를 만나보세요. 다음뉴스는 국내외 주요이슈와 실시간 속보, 문화생활 및 다양한 분야의 뉴스를 입체적으로 전달하고 있습니다.\nLG전자 베스트샵 서울양평점이 내년 1월 5일 'LG전자 베스트샵 서울양평220점'으로 새롭게 오픈한다고 21일 밝혔다.\n이에 LG전자 베스트샵 서울양평220점에서는 다양한 사전 행사를 진행한다. 지난 15일부터 시작된 사전 예약 기간동안 LG전자 가전제품 동시 구매 시 최대 460만 혜택을 제공하는 이벤트를 진행한다.

In [80]:

from langchain_core.prompts.chat import ChatPromptTemplate
template = """
Utilizing the context given below, answer the question.
[context]
{context}

question: {query} is good for invest?
answer korean
"""
prompt = ChatPromptTemplate.from_template(template)

In [81]:
chain = RunnableParallel({"context":retriever, "query":RunnablePassthrough()})\
    | prompt\
    | llm \
    | StrOutputParser()

In [82]:
chain.invoke("SK하이닉스")

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'네, SK하이닉스는 투자하기에 좋은 종목으로 보입니다. 여러 문서에서 SK하이닉스의 주가가 강세를 보이고 있으며, 특히 AI 랠리를 이끄는 엔비디아의 핵심 공급망 업체로 꼽히면서 주가가 상승하고 있다는 점이 강조되고 있습니다. 또한, SK하이닉스는 2024년 5월 23일 기준으로 주당 20만원을 돌파하며 지속적인 상승세를 보이고 있습니다. 증권가에서도 SK하이닉스 주가가 앞으로 추가 상승할 것이라는 긍정적인 전망을 내놓고 있습니다. 따라서 SK하이닉스는 투자하기에 좋은 종목으로 판단됩니다.'

In [88]:
def dummy(anything):
    return 'this is dummy'

In [89]:
chain = RunnableParallel(context=retriever, query=RunnablePassthrough(), dummy=dummy)\
    | prompt\
    | llm \
    | StrOutputParser()

In [90]:
chain.invoke("삼성전자")

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


"삼성전자는 투자하기에 좋은 선택일 수 있습니다. 최근 삼성전자의 주가는 52주 최고가를 경신하며 상승세를 보이고 있습니다. 메리츠증권은 삼성전자의 목표주가를 기존 9만4000원에서 9만5000원으로 상향 조정하고, 투자의견을 '매수'로 유지했습니다. 이는 삼성전자의 영업이익이 개선될 것으로 전망되기 때문입니다. 따라서 현재 상황을 고려할 때, 삼성전자는 투자하기에 긍정적인 선택일 수 있습니다."

In [94]:
chain = RunnableParallel(context=retriever|merge_docs, query=RunnablePassthrough(), dummy=dummy)\
    | prompt \
    | llm \
    | StrOutputParser()
chain.invoke("삼성전자")

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


"네, 삼성전자는 투자하기에 좋은 선택일 수 있습니다. 메리츠증권은 삼성전자에 대한 목표주가를 기존 9만4000원에서 9만5000원으로 상향 조정하며, 투자의견을 '매수'로 유지했습니다. 이는 삼성전자의 영업이익이 개선될 것으로 전망되기 때문입니다. 특히, 반도체(DS) 부문의 영업손실이 감소하고, D램 감산폭을 줄이는 등의 긍정적인 요인들이 반영된 결과입니다. 따라서 삼성전자는 투자 가치가 있는 종목으로 평가될 수 있습니다."

In [95]:
from langchain import hub

In [98]:
prompt = hub.pull("rlm/rag-prompt")

In [99]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])